In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [2]:
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 2.5.1+cu121


In [3]:
DATA_DIR = "/kaggle/input/bank-note-authentication-uci-data/BankNote_Authentication.csv"
data = pd.read_csv(DATA_DIR)

In [4]:
# splitting features and classes
X, y = data.iloc[:, :-1], data.iloc[:, -1]

In [5]:
# converting data to numpy arrays for comptational reasons
X = X.to_numpy()
y = y.to_numpy().reshape(-1, 1)

In [6]:
# splitting data to train and test
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42, stratify=y)

In [7]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [8]:
torch.manual_seed(42)

In [9]:
class FinalMLP(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(FinalMLP, self).__init__()
        self.W1 = nn.Parameter(torch.randn(input_size, hidden_size, requires_grad=True))
        self.b1 = nn.Parameter(torch.randn(1 , hidden_size, requires_grad=True))
        self.W2 = nn.Parameter(torch.randn(hidden_size, out_size, requires_grad=True))
        self.b2 = nn.Parameter(torch.randn(1, out_size, requires_grad=True))

    def forward(self, X):
        self.z1 = torch.matmul(X, self.W1) + self.b1
        self.a1 = torch.tanh(self.z1)
        self.z2 = torch.matmul(self.a1, self.W2) + self.b2
        self.a2 = torch.sigmoid(self.z2)
        return self.a2

    def train(self, X, y, epochs=5000, lr=1e-2):
        criterion = nn.BCELoss()
        optimizer = optim.SGD(self.parameters(), lr=lr)
        losses = []
        for epoch in range(epochs):
            optimizer.zero_grad()
            output = self.forward(X)
            loss = criterion(output, y.float())
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
            if (epoch % 500 == 0):
                print(f"Epoch [{epoch}/{epochs}], Loss: {loss.item():.4f}")
        print(f"Epoch [{epochs}/{epochs}], Loss: {loss.item():.4f}")

    def predict(self, X):
        with torch.no_grad():
            preds = model.forward(X)
            preds = (preds > 0.5).float()
            return preds

In [10]:
input_size = X_train.shape[1]
hidden_size = 5
out_size = 1   # for binary classification
model = FinalMLP(input_size, hidden_size, out_size)

In [11]:
model.train(X_train, y_train)

Epoch [0/5000], Loss: 1.0897
Epoch [500/5000], Loss: 0.2211
Epoch [1000/5000], Loss: 0.1274
Epoch [1500/5000], Loss: 0.0839
Epoch [2000/5000], Loss: 0.0628
Epoch [2500/5000], Loss: 0.0497
Epoch [3000/5000], Loss: 0.0414
Epoch [3500/5000], Loss: 0.0357
Epoch [4000/5000], Loss: 0.0318
Epoch [4500/5000], Loss: 0.0288
Epoch [5000/5000], Loss: 0.0266


In [12]:
preds = model.predict(X_test)

In [13]:
def print_metrics(y_test, y_pred):
    y_test = y_test.flatten()
    y_pred = y_pred.flatten()

    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average="binary")
    recall = recall_score(y_test, y_pred, average="binary")
    f1 = f1_score(y_test, y_pred, average="binary")
    conf_mx = confusion_matrix(y_test, y_pred)

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("\nConfusion Matrix:")
    print(conf_mx)
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred)) 

In [15]:
print_metrics(y_test, preds)

Accuracy: 0.9964
Precision: 0.9919
Recall: 1.0000
F1 Score: 0.9959

Confusion Matrix:
[[152   1]
 [  0 122]]

Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00       153
         1.0       0.99      1.00      1.00       122

    accuracy                           1.00       275
   macro avg       1.00      1.00      1.00       275
weighted avg       1.00      1.00      1.00       275

